## Mandar a Predecir Datos

Para no saturar la consulta se manda a predecir desde la hoja "predecir", esta funcion recorre resgistros por fila y consulta mediante loop

* Se debe correr sas 0 generacion datos ODD_V2.
* se deben tener los paquetes os, pandas, xlrd y request para la lectura de excel

In [ ]:
#Carga de paquetes
import os
import pandas as pd
import requests

## carga de datos

In [108]:
mainpath = "/Users/salibaud/Desktop/python/kaufmann/ESTIMADOR DE KML AUTOMOVILES"
filename = "Mnadar a predecir.xlsx"
fullpath = os.path.join(mainpath, filename)
xl = pd.ExcelFile(fullpath)
xl
data = xl.parse('predecir')
data.head()

,RUT,NRO_CHASSIS,PROVINCIA_CLIENTE,FORMA_DE_PAGO,FAMILIA,MODELO,SEXO_CLIENTE,MONTO_VENTA,MONEDA,TIPO_CAMBIO,MODELO_DESCRIPCION
0,10003652-5,WDDBF4EB9FJ336330,SANTIAGO,FINANC. EXTERNO,NGCC,CLASE A,Masculino,1.749000e+07,PESO,600.37,A 250
1,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180
2,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180
3,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180
4,10012322-3,WDC2923241A091039,CHACABUCO,NORMAL,SUV,CLASE E,Femenino,6.758240e+04,DOLAR KAUFMANN,634.82,GLE 350 D COUPE


## Llamada a la api

In [110]:

login = {
    'username': 'kaufmann_api',
    'password': 'BpZgb74qtk',
}


resultados_modelo=[]

i=0

for i in range(len(data)):
    params=data.iloc[i].to_dict()
    results = requests.post('https://api.odd.co/clairvoyant/jwt-auth/', json=login)
    token = results.json()['token']
    headers = {'Authorization': 'JWT ' + token}
    results = requests.post('https://api.odd.co/clairvoyant/kaufmann/kml/',headers=headers,json=params)
    print("predicción numero ",i)
    x=results.json()
    resultados_modelo.append(x)




## Union de resultados con datos originales

In [112]:
df = pd.DataFrame(resultados_modelo)
a=data.join(df)
a.head()

,RUT,NRO_CHASSIS,PROVINCIA_CLIENTE,FORMA_DE_PAGO,FAMILIA,MODELO,SEXO_CLIENTE,MONTO_VENTA,MONEDA,TIPO_CAMBIO,MODELO_DESCRIPCION,predictions
0,10003652-5,WDDBF4EB9FJ336330,SANTIAGO,FINANC. EXTERNO,NGCC,CLASE A,Masculino,1.749000e+07,PESO,600.37,A 250,[845.7146781035044]
1,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180,[741.926623604907]
2,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180,[741.926623604907]
3,10004237-1,WDD1173421N541095,SANTIAGO,NORMAL,NGCC,CLASE CLA,Masculino,1.923280e+07,PESO,633.72,CLA 180,[741.926623604907]
4,10012322-3,WDC2923241A091039,CHACABUCO,NORMAL,SUV,CLASE E,Femenino,6.758240e+04,DOLAR KAUFMANN,634.82,GLE 350 D COUPE,[3585.3077043532726]


## exportacion de archivo

Una vez generada la consulta, se manda a exportar a excel el listado

In [113]:
a.to_csv('resultados_prediccion.csv')